In [75]:
import os

import pandas as pd
from Dani_scripts.ELM_pipeline import scoring
import numpy as np

def make_alpha_df(high_alpha, low_alpha):
    pred_probs_emo_1 = pd.read_csv(os.path.join(high_alpha, "best_preds_emo.csv")).to_numpy()
    pred_probs_aro_val_1 = pd.read_csv(os.path.join(high_alpha, "best_preds_aro_val.csv")).to_numpy()
    pred_probs_emo_2 = pd.read_csv(os.path.join(low_alpha, "best_preds_emo.csv")).to_numpy()
    pred_probs_aro_val_2 = pd.read_csv(os.path.join(low_alpha, "best_preds_aro_val.csv")).to_numpy()
    alpha_df = pd.DataFrame(columns=['alpha', 'mean_ccc_emo', 'mean_ccc_aroval'])
    for alpha in np.arange(0.0, 1.1, 0.1):
        pred_probs_emo = alpha * pred_probs_emo_1 + (1 - alpha) * pred_probs_emo_2
        pred_probs_aro_val = alpha * pred_probs_aro_val_1 + (1 - alpha) * pred_probs_aro_val_2
        ccc_emo, _ = scoring("ELM", y_test_emo, pred_probs_emo)
        ccc_aroval, _ = scoring("ELM", y_test_aroval, pred_probs_aro_val)
        mean_ccc_emo = np.round(np.mean(ccc_emo), 4)
        mean_ccc_aroval = np.round(np.mean(ccc_aroval), 4)
        alpha_df.loc[len(alpha_df)] = [alpha, mean_ccc_emo, mean_ccc_aroval]
    return alpha_df

y_test_emo = pd.read_csv(r"Dani_scripts/data/labels_csv/high_info.csv")
y_test_aroval = pd.read_csv(r"Dani_scripts/data/labels_csv/two_info.csv")
y_test_emo = y_test_emo[y_test_emo['Split'] == 'Val'].sort_values(by='File_ID').reset_index(drop=True).drop(["File_ID", "Split"], axis=1).to_numpy()
y_test_aroval = y_test_aroval[y_test_aroval['Split'] == 'Val'].sort_values(by='File_ID').reset_index(drop=True).drop(["File_ID", "Split"], axis=1).to_numpy()

high_alpha = os.path.join("results", "exp_batch11_compare_funcs_nonlinearkernels")
low_alpha = os.path.join("results", "exp_batch20_words_compare_llds_130pca_200gmm_fv_plus_compare_funcs_nonlinearkernels")

alpha_df = make_alpha_df(high_alpha, low_alpha)
best_CCC_emo = alpha_df.sort_values(by='mean_ccc_emo', ascending=False).iloc[0]['mean_ccc_emo']
best_CCC_aroval = alpha_df.sort_values(by='mean_ccc_aroval', ascending=False).iloc[0]['mean_ccc_aroval']
best_alpha = alpha_df[alpha_df['mean_ccc_emo'] == best_CCC_emo].iloc[0]['alpha']
# best_preds = best_alpha * pred_probs_1 + (1 - best_alpha) * pred_probs_2

print(alpha_df[['mean_ccc_emo', 'mean_ccc_aroval']].transpose().to_string(index=False))
print(best_alpha, best_CCC_emo, best_CCC_aroval)

     0       1       2       3       4       5       6       7       8       9       10
 0.4941  0.4998  0.5045  0.5082  0.5109  0.5125  0.5130  0.5124  0.5108  0.5081  0.5044
 0.4802  0.4863  0.4912  0.4950  0.4975  0.4986  0.4985  0.4971  0.4944  0.4906  0.4856
0.6000000000000001 0.513 0.4986


,Awe,Excitement,Amusement,Awkwardness,Fear,Horror,Distress,Triumph,Sadness,Surprise
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
19391,False,False,False,False,False,False,False,False,False,False
19392,False,False,False,False,False,False,False,False,False,False
19393,False,False,False,False,False,False,False,False,False,False
19394,False,False,False,False,False,False,False,False,False,False
